In [1]:
# ---- Libraries -----

from fenics import *
import numpy as np
import matplotlib.pyplot as plt
import math
from dolfin import *
!pip install scipy
import scipy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
# ---- Parameters-----

# File name
#filename = "test1"    # [0.01, 0], [0, 0.01]]        # p=1
#filename = "test2"    # [0.01, 0], [0, 0.01]]        # p=2
#filename = "test3"    # [[0.01, 0.01], [0, 0.01]]    # p=1
filename = "test4"     [[0.01, 0.01], [0, 0.01]]    # p=2


# Diffusion (choose the coefficients according to the test you want to run)
d11 = 0.01
d12 = 0.01 
d21 = 0
d22 = 0.01
components_D = [[d11, d12], [d21, d22]]

# Time
t0 = 0
dt = 1e-5
num_steps = 10
T = num_steps*dt

# Functional space
p = 2              # polynomial degree 

# Stabilization
alpha = 3

# Initial datum
u0 = Expression('(1-cos(2*pi*x[0]))*(1-cos(2*pi*x[1]))', degree = p)


# Exact solution
u_ex = Expression('exp(-t)*(1-cos(2*pi*x[0]))*(1-cos(2*pi*x[1]))', t = 0, degree = p)

# Force
f = Expression('exp(-t)*(exp(-t)*(cos(2*pi*x[0]) - 1)*(cos(2*pi*x[1]) - 1) - 1)*(cos(2*pi*x[0]) - 1)*(cos(2*pi*x[1]) - 1) - exp(-t)*(cos(2*pi*x[0]) - 1)*(cos(2*pi*x[1]) - 1) + d11*pi*pi*exp(-t)*cos(2*pi*x[0])*(cos(2*pi*x[1]) - 1) + 4*d22*pi*pi*exp(-t)*cos(2*pi*x[1])*(cos(2*pi*x[0]) - 1) - d12*pi*pi*exp(-t)*sin(2*pi*x[0])*sin(2*pi*x[1]) - d21*pi*pi*exp(-t)*sin(2*pi*x[0])*sin(2*pi*x[1])', d11 = d11, d12 = d12, d21 = d21, d22 = d22, t = 0, degree = p)

# Neumann boundary condition
gN = Expression('0*x[0] + 0*x[1]', degree = p)

In [3]:
# ---- Computations ----

# Diffusion tensor
D = as_matrix(components_D)
norm_D = np.linalg.norm(np.matrix(components_D))

In [4]:
# ---- Convergence test initialization ----

N = np.power(2,[2,3,4,5,6,7])
err_H1 = np.zeros(len(N))
err_L2 = np.zeros(len(N))

In [5]:
for i in range(1,len(N)):

    # ---- Geometry ----

    nx = ny = N[i]

    # Create mesh and define function space
    hx = 1/nx
    hy = 1/ny
    h = sqrt(hx*hx + hy*hy)     # square's diagonal (diam(elem))
    mesh = UnitSquareMesh(nx, ny)
    V = FunctionSpace(mesh, "DG", p)

    # DG stabilization coefficient
    sigma = norm_D + p*p/h*alpha

    # Projection of u_0 on V
    u0h = interpolate(u0, V)

    # ---- Variational problem ----

    # Solution
    u_n = u0h
    u = TrialFunction(V)

    # Test function
    v = TestFunction(V)

    # Outer normal
    n = FacetNormal(mesh)

    # DG bilinear form
    B = inner(D*grad(u), grad(v))*dx - inner(avg(D*grad(u)), jump(v, n))*dS -\
        inner(avg(D*grad(v)), jump(u, n))*dS +\
        sigma*inner(jump(u, n), jump(v, n))*dS

    # Final problem
    a = u*v*dx + dt*B - dt*(1-u_n)*u*v*dx   # reaction coeff = 1 
    F = u_n*v*dx + gN*v*ds + dt*f*v*dx

    # ---- Time-stepping ----

    u = Function(V)

    # Initial time
    t = t0
    
    # Create VTK file for saving solution
    vtkfile = File('solutions_convergence/' + filename + '/' + filename +'.pvd')
    
    # Plot initial solution 
    #plot(u_n)
    #plt.show()
    
    for n in range(num_steps):
        
        # Update current time
        t += dt

        # Plot
        #plot(u_n)
        #plt.show()
        
        # Compute solution
        solve(a == F, u)
          
        # Update previous solution
        u_n.assign(u)
        
        # Save

        vtkfile << (u_n, t)
        

    # Plot
    #plot(u_n)
    #plt.show()
    err_H1[i] = errornorm(u_ex, u, 'H1')
    err_L2[i] = errornorm(u_ex, u, 'L2')
       

Calling FFC just-in-time (JIT) compiler, this may take some time.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
*** Warning: Degree of exact solution may be inadequate for accurate result in errornorm.
*** Warning: Degree of exact solution may be inadequate for accurate result in errornorm.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solv

In [6]:
# Computation of the error rates 
h = np.sqrt(1/N*1/N + 1/N*1/N ) 
order_H1 = np.log(err_H1[1:(len(err_H1)-1)]/err_H1[2:len(err_H1)]) / np.log(h[1:(len(h)-1)]/h[2:len(h)])
order_L2 = np.log(err_L2[1:(len(err_L2)-1)]/err_L2[2:len(err_L2)]) / np.log(h[1:(len(h)-1)]/h[2:len(h)])

array([0.35355339, 0.1767767 , 0.08838835, 0.04419417, 0.02209709,
       0.01104854])